# Advanced parking lot detection system
Down below, we're gonna teach a Support Vector Machine to recognize when a parking spot is empty.
Our clever plan involves using a croner or an edge detection algorithm to find corners and edges in different training examples, summing them.<br>
The idea is that if there's a car present, there will be more detections, resulting in a higher sum value.
In the .csv file, we'll use the occupancy field as our label. As for the parking spots themselves, they're represented by three features: the sum of pixel values after applying the algorithm, plus the weather and the hour when the image was taken.


In [ ]:
import cv2
import pickle
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Import the dataset
path_to_data = "CNRPark+EXT.csv"
data = pd.read_csv(path_to_data, low_memory=False)

# Load image urls
split = 12584
image_url = data["image_url"].to_numpy()

# Create the trainset
trainset = np.empty((data.shape[0],3), dtype="object")

# Feature number zero is the weather
def map_weather(w):
    return 0 if w == "S" else (1 if w == "R" else 2)

weather = data["weather"].to_numpy()
weather_mapper = np.vectorize(map_weather)
trainset[:,0] = weather_mapper(weather)

# Feature number one is the hour
trainset[:,1] = data["hour"].to_numpy()
label = data["occupancy"].to_numpy()

In [ ]:
def get_img_from_url(image):
    return image if isinstance(image, np.ndarray) else cv2.imread(image, cv2.IMREAD_GRAYSCALE)


# Define a function for each different Edge and Corner Detector
def canny_edge_count(image_url):
    image = get_img_from_url(image_url)
    # image = cv2.equalizeHist(image)
    
    lt, ht, size = 50, 200, 5
    edges = cv2.Canny(image, lt, ht, size)
    
    return np.sum(edges) / (edges.shape[0] * edges.shape[1])

def harris_corner_count(image_url):
    # Performing a GaussianBlur to remove noises and small structures
    image = get_img_from_url(image_url)
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)
    
    corners = cv2.cornerHarris(blurred_image, 2, 3, 0.04)
    
    return corners.sum()

def susan_corner_count(image_url):
    # Performing a GaussianBlur to remove noises and small structures
    image = get_img_from_url(image_url)
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)

    susan = cv2.ximgproc.createFastFeatureDetector()
    keypoints = susan.detect(blurred_image)
    
    return len(keypoints)

def fast_corner_count(image_url):
    # Performing a GaussianBlur to remove noises and small structures
    image = get_img_from_url(image_url)
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)
    
    fast = cv2.FastFeatureDetector_create()
    keypoints = fast.detect(blurred_image, None)
    
    return len(keypoints)

In [ ]:
# Saving the canny edge count on the disk for reuse
canny_mapper = np.vectorize(canny_edge_count)
canny_edges = canny_mapper(image_url)

# Saving the harris corner count on the disk for reuse
harris_mapper = np.vectorize(harris_corner_count)
harris_corners = harris_mapper(image_url)

# Saving the susan corner count on the disk for reuse
susan_mapper = np.vectorize(susan_corner_count)
susan_corners = None

# Saving the fast corner count on the disk for reuse
fast_mapper = np.vectorize(harris_corner_count)
fast_corners = fast_mapper(image_url)

with open("edge_corner_count.pkl", 'wb') as file:  
    pickle.dump([canny_edges, harris_corners, susan_corners, fast_corners], file)

In [ ]:
with open("edge_corner_count.pkl", 'rb') as file:  
    canny_edges, harris_corners, susan_corners, fast_corners = pickle.load(file)

# Feature number two is the edge or corner count
trainset[:,2] = harris_corners

In [ ]:
# Prepare the data for the fit
X = trainset    # Trainset is the X
y = label       # Labels are the y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Normalize the features
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the SVM model
model = svm.SVC()
model.fit(X_train_scaled, y_train)

# Evaluate the model
accuracy = model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy * 100, "%")

In [ ]:
# Save the model on the disk
with open("svm_harris_model.pkl", 'wb') as file:  
    pickle.dump(model, file)

# Fullsize image testing
Algorithm testing with high qquality parking images provided by the CNR

In [ ]:
from os import path

# Select a random full image
random_index = int(split + (split - data.shape[0]) * np.random.rand())
img_descriptor = data.iloc[random_index]

# Compute the path of the fullsize image
resized_path = "CNR-EXT/PATCHES"
fullsize_path = "CNR-EXT_FULL_IMAGE_1000x750/FULL_IMAGE_1000x750"

filename = ("%d-%02d-%02d_%02d%02d.jpg" % (img_descriptor["year"], img_descriptor["month"], img_descriptor["day"], 
                                           img_descriptor["hour"], img_descriptor["minute"]))


image_path = img_descriptor["image_url"].replace(resized_path, fullsize_path)
image_path = path.dirname(image_path)
image_path = path.join(image_path, filename)

# Import and resize the image
img = cv2.imread(image_path)
img = cv2.resize(img, (2592,1944), interpolation=cv2.INTER_LINEAR)

# Convert to grayscale to perform the algorithm on it
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Import the csv file corrisponding to the right camera
camera_csv = ("CNR-EXT_FULL_IMAGE_1000x750/camera%01d.csv" % int(img_descriptor["camera"]))
camera_descriptor = pd.read_csv(camera_csv)

# Create a variable containing data to be predicted
slot = np.empty((camera_descriptor.shape[0],3), dtype="object")

# Map the weather
slot[:,0] = map_weather(img_descriptor["weather"])

# Insert the hour
slot[:,1] = img_descriptor["hour"]

# Get slots informations
x, y, w, h = camera_descriptor.values[:,1], camera_descriptor.values[:,2], camera_descriptor.values[:,3], camera_descriptor.values[:,4]

# Perform the edge or corner detection on those boundaries
for i in range(camera_descriptor.shape[0]):
    slot_img = gray_img[y[i] : y[i] + h[i], x[i] : x[i] + w[i]]
    slot[i,2] = harris_corner_count(slot_img)

# Perform the prediction
slot = scaler.transform(slot)
result = model.predict(slot)

# Draw rectangles in park slots
for i in range(len(result)):
    if result[i]==0:
        colore=(0,255,0)
    else:
        colore=(0,0,255)
    cv2.rectangle(img, (x[i], y[i]), (x[i] + w[i], y[i] + h[i]), colore, 2)

# Display the image
img = cv2.resize(img, (1000,750), interpolation=cv2.INTER_LINEAR)
cv2.imshow("Parcheggi", img)
cv2.waitKey(0)
cv2.destroyAllWindows()